In [6]:
# Neo4j Graph Query Examples
from neo4j import GraphDatabase

# Initialize Neo4j driver
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "none"))

print("=== NEO4J GRAPH QUERIES ===\n")

with neo4j_driver.session() as session:
    
    # 1. Get basic statistics
    print("1. DATABASE STATISTICS:")
    stats_query = """
    MATCH (p:Page) 
    OPTIONAL MATCH (p)-[:HAS_CHUNK]->(c:Chunk)
    OPTIONAL MATCH (p)-[:HAS_IMAGE]->(i:Image)
    RETURN 
        count(DISTINCT p) as total_pages,
        count(DISTINCT c) as total_chunks,
        count(DISTINCT i) as total_images
    """
    result = session.run(stats_query)
    stats = result.single()
    print(f"   Pages: {stats['total_pages']}")
    print(f"   Chunks: {stats['total_chunks']}")
    print(f"   Images: {stats['total_images']}")
    
    # 2. Show hierarchical structure (parent-child relationships)
    print("\n2. HIERARCHICAL STRUCTURE:")
    hierarchy_query = """
    MATCH (parent:Page)-[:HAS_CHILD]->(child:Page)
    RETURN parent.title as parent_title, 
           collect(child.title) as children
    ORDER BY parent.title
    LIMIT 5
    """
    results = session.run(hierarchy_query)
    for record in results:
        print(f"   📁 {record['parent_title']}")
        for child in record['children'][:3]:  # Show first 3 children
            print(f"      └── {child}")
        if len(record['children']) > 3:
            print(f"      └── ... and {len(record['children']) - 3} more")
    
    # 3. Find pages about specific topics
    print("\n3. PAGES ABOUT DOSE RATE:")
    topic_query = """
    MATCH (p:Page)
    WHERE toLower(p.title) CONTAINS 'dose' OR toLower(p.title) CONTAINS 'rate'
    RETURN p.title as title, p.url as url
    ORDER BY p.title
    """
    results = session.run(topic_query)
    for record in results:
        print(f"   📄 {record['title']}")
        print(f"      🔗 {record['url']}")
    
    # 4. Find pages with most chunks (longest content)
    print("\n4. PAGES WITH MOST CONTENT:")
    content_query = """
    MATCH (p:Page)-[:HAS_CHUNK]->(c:Chunk)
    WITH p, count(c) as chunk_count
    RETURN p.title as title, chunk_count, p.text_length as text_length
    ORDER BY chunk_count DESC
    LIMIT 5
    """
    results = session.run(content_query)
    for record in results:
        print(f"   📄 {record['title']}")
        print(f"      📦 {record['chunk_count']} chunks, {record['text_length']} chars")
    
    # 5. Find root pages (no parents)
    print("\n5. ROOT PAGES (Top-level):")
    root_query = """
    MATCH (p:Page)
    WHERE NOT (p)-[:HAS_PARENT]->()
    RETURN p.title as title, p.url as url
    ORDER BY p.title
    """
    results = session.run(root_query)
    for record in results:
        print(f"   🏠 {record['title']}")
    
    # 6. Find leaf pages (no children)
    print("\n6. LEAF PAGES (No children):")
    leaf_query = """
    MATCH (p:Page)
    WHERE NOT (p)-[:HAS_CHILD]->()
    RETURN p.title as title
    ORDER BY p.title
    LIMIT 10
    """
    results = session.run(leaf_query)
    leaf_titles = [record['title'] for record in results]
    for title in leaf_titles:
        print(f"   📜 {title}")
    
    # 7. Graph traversal: Find all descendants of a specific page
    print("\n7. DESCENDANTS OF 'Portal:SwissFEL':")
    descendants_query = """
    MATCH path = (root:Page {title: 'Portal:SwissFEL'})-[:HAS_CHILD*]->(descendant:Page)
    WITH descendant, length(path) as depth
    RETURN descendant.title as title, depth
    ORDER BY depth, title
    LIMIT 10
    """
    results = session.run(descendants_query)
    for record in results:
        indent = "  " * record['depth']
        print(f"   {indent}└── {record['title']} (depth: {record['depth']})")
    
    # 8. Complex query: Pages with images but no children (detailed leaf pages)
    print("\n8. DETAILED LEAF PAGES (with images, no children):")
    detailed_query = """
    MATCH (p:Page)-[:HAS_IMAGE]->(i:Image)
    WHERE NOT (p)-[:HAS_CHILD]->()
    WITH p, count(i) as image_count
    RETURN p.title as title, image_count
    ORDER BY image_count DESC
    LIMIT 5
    """
    results = session.run(detailed_query)
    for record in results:
        print(f"   🖼️  {record['title']} ({record['image_count']} images)")

# Close the driver
neo4j_driver.close()
print("\n✅ Neo4j queries completed!")

=== NEO4J GRAPH QUERIES ===

1. DATABASE STATISTICS:
   Pages: 50
   Chunks: 200
   Images: 124

2. HIERARCHICAL STRUCTURE:
   📁 Portal:SwissFEL
      └── Portal:SwissFEL/SwissFEL_Prozeduren_und_Checklisten
      └── Portal:SwissFEL/SwissFEL_Injector
      └── Portal:SwissFEL/SwissFEL_Undulators
   📁 Portal:SwissFEL/SwissFEL_Injector
      └── SF_RF_Phasing
      └── SwissFEL_Einführung_(Anfänger)#Die_Elektronenquelle_.2F_Gun_.28SINEG01.29
      └── SwissFEL_Einführung_(Anfänger)#Gun_Spektrometer_(SINBD01)
      └── ... and 5 more
   📁 Portal:SwissFEL/SwissFEL_Prozeduren_und_Checklisten
      └── Laser-Gun-Setup_Routine
      └── SwissFEL_Energiewechsel
      └── Switchyard#Bedienung
      └── ... and 18 more
   📁 Portal:SwissFEL/SwissFEL_Undulators
      └── Undulator_Pointing_Anpassung
      └── Undulator
      └── Beschleunigerphysik_SLS#Synchrotronstrahlung
      └── ... and 4 more
   📁 None
      └── Notfall_Prozedur_SwissFEL
      └── SF_Schichtaufgaben
      └── SwissFEL_Einführ

In [5]:
# Quick test with updated API
from qdrant_client import QdrantClient
from embeddings import get_embedder

# Initialize
qdrant_client = QdrantClient(url="http://localhost:6333")
embedder = get_embedder()

# Check collection status
collections = qdrant_client.get_collections()
collection_names = [col.name for col in collections.collections]
print(f"Collections: {collection_names}")

if "swissfel_wiki" in collection_names:
    collection_info = qdrant_client.get_collection("swissfel_wiki")
    print(f"Points in collection: {collection_info.points_count}")
    
    # Simple search test
    query = "What happened to the DRM?"
    query_vector = embedder.encode_query(query)
    
    # Use updated API with no threshold first
    search_results = qdrant_client.query_points(
        collection_name="swissfel_wiki",
        query=query_vector[0].tolist(),
        limit=3,
        score_threshold=0.0,  # No threshold
        with_payload=True
    )
    
    print(f"\nFound {len(search_results.points)} results:")
    for i, result in enumerate(search_results.points, 1):
        print(f"{i}. Score: {result.score:.4f}")
        print(f"   Title: {result.payload.get('title', 'N/A')}")
        print(f"   Text: {result.payload.get('text', '')[:200]}...")
        print()
else:
    print("Collection not found!")

Collections: ['swissfel_wiki']
Points in collection: 200

Found 3 results:
1. Score: 0.3715
   Title: Notfall_Prozedur_SwissFEL
   Text: sollten alle informiert sein. • Der Vorfall sollte zeitnah ausführlich im Elog zusammengefasst werden (entweder vom RC oder dem Maschinenkoordinator) Unterbrüche während Maschinenschichten (neu auch m...

2. Score: 0.3213
   Title: AFS_Tipps_and_Tricks
   Text: with kdestroy. This does not remove the AFS Token, so I still have access. But now the AFS token cannot be renewed any longer. So when I do unlog now, I cannot regain it with aklog. Now the interestin...

3. Score: 0.2966
   Title: BCM/CDR
   Text: wiederum verglichen, usw. Troubleshooting S10MA01-DCDR080 - im Tunnel installierte Hardware CDR Status bleibt in irgend einem Zustand hängen, obwohl alle LEDs oben links grün sind: -> Klicke "STOP" un...

